Developing a deep fake filter model in Python. To create a deep fake filter model in Python, you will be working with OpenCV to open a camera window and apply a filter featuring Cristiano Ronaldo or any other celebrity onto the video feed.

In [5]:
import dlib
import cv2
import numpy as np

print(dlib.__version__)  # to check if dlib is properly installed

def apply_filter(frame, celebrity_img, detector, predictor):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_frame)

    if len(faces) == 0:
        print("No faces detected")
        return frame

    for face in faces:
        landmarks = predictor(gray_frame, face)
        points = [(p.x, p.y) for p in landmarks.parts()]

        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_img = frame[y:y+h, x:x+w]

        # Resize and align the celebrity image to the face landmarks
        celebrity_resized, mask = align_and_resize(celebrity_img, points, (w, h))

        # Ensure all images and masks are the same size
        face_img = cv2.resize(face_img, (w, h))
        mask = cv2.resize(mask, (w, h))

        # Blend the celebrity face with the detected face region
        blended_face = blend_faces(face_img, celebrity_resized, mask)

        # Replace the face region with the blended face
        frame[y:y+h, x:x+w] = blended_face

    return frame

def align_and_resize(celebrity_img, points, face_size):
    # Points for aligning the celebrity image (assuming similar landmarks)
    celeb_points = np.array([
        (30, 50), (50, 50), (40, 70), (35, 90), (45, 90)
    ], dtype=np.float32)

    # Get corresponding points from detected face landmarks
    face_points = np.array([
        points[36], points[45], points[30], points[48], points[54]
    ], dtype=np.float32)

    # Calculate the transformation matrix
    matrix = cv2.getAffineTransform(celeb_points[:3], face_points[:3])

    # Apply the affine transformation to celebrity image
    aligned_celeb = cv2.warpAffine(celebrity_img, matrix, (celebrity_img.shape[1], celebrity_img.shape[0]))

    # Resize aligned celebrity image and create mask for blending
    aligned_celeb_resized = cv2.resize(aligned_celeb, face_size)
    mask = np.zeros((face_size[1], face_size[0], aligned_celeb.shape[2]), dtype=np.uint8)
    cv2.fillConvexPoly(mask, (face_points - face_points.min(axis=0)).astype(int), (255,) * aligned_celeb.shape[2])

    return aligned_celeb_resized, mask

def blend_faces(face_img, celebrity_img, mask):
    # Convert mask to float for blending
    mask = mask.astype(float) / 255
    inv_mask = 1 - mask

    # Ensure the mask is the same size as the face image
    mask = cv2.resize(mask, (face_img.shape[1], face_img.shape[0]))
    inv_mask = cv2.resize(inv_mask, (face_img.shape[1], face_img.shape[0]))

    # Blend the images using the mask
    blended_face = (face_img * inv_mask + celebrity_img * mask).astype(np.uint8)

    return blended_face

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/rahul/Downloads/archive (27)/shape_predictor_68_face_landmarks.dat")

celebrity_img = cv2.imread("C:/Users/rahul/Downloads/ronaldo.jpg")
celebrity_img = cv2.cvtColor(celebrity_img, cv2.COLOR_BGR2RGB)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Camera not accessible")
else:
    print("Camera opened successfully")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame = apply_filter(frame, celebrity_img, detector, predictor)
    cv2.imshow("Deep Fake Filter", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


19.24.0
Error: Camera not accessible
Failed to grab frame
